# Computing Radiative Capture Rates


## Introduction

The theory discussed here can be found in the following paper:

```
Dreyer, C. E., Alkauskas, A., Lyons, J. L., & Van de Walle, C. G. (2020). Radiative capture rates at deep defects from electronic structure calculations. Phys Rev B, 102(8), 085305. doi: 10.1103/PhysRevB.102.085305
```

The theory will be presented only for radiative capture but will be directly applicable to photo-excitation.


The cature of an electron from the conduction band can be considered as the following process:

$A^{0} + e^{-} \rightarrow A^{-}$ .

The radiative capture rate is given by:

$R_{n} = C_{n}N_A^0n$ ,

where $N_A^0$ is the concentraion of the neutral defect, n is the electron carrier concentration.

A schematic of the process can be seen on the configuration-coordinate diagram:

<img src="https://github.com/materialsproject/pymatgen-analysis-defects/blob/main/docs/source/_static/img/rad_capture_1.png?raw=true" width="600" />



## Derivation


If we assume that the electronic and vibronic wavefuntion is sperable (Born-Oppenheimer) approximation.

$\mathbf\Psi(\mathbf{Q}, \mathbf{x}) = \Psi(\mathbf{x})\chi(\mathbf{\mathbf Q})$ ,

where:
   - $\Psi$: the electronic wavefunction.
   - $\chi$: The ionic wavefunction.
   

For $e^-$ capture the initial state is $A^0 + e^-$, and for simplicity let's just first assume that the initial states is the ground state of the simple harmonic oscillator (SHO) represented by the upper parabola.

The optical transition is given by the momentum operator

$ \hat{\mathbf P} = -i \hbar \sum_j \frac{\partial}{\partial \mathbf{x}_j} $

<div class="alert alert-block alert-warning"> 
    Trust me bro moment: if you assume that the Many-body wavefunction is a slater-determinant then the manybody matrix element will essentially see everything but the changing electron cancel.  So you can just replace the many-body electronic matrix element with the single particle matrix element $p_{if}$.
</div>

The luminescence intensity is given by:

$ I(\hbar \omega) = \frac{e^2 n_r \eta_{\rm sp} \omega}{3 m^2 \epsilon_0 \pi c^3 \hbar} \left| p_{if} \right|^2 \sum_n \left| \left< \chi_{i0} | \chi_{fn}  \right> \right|^2 \delta(E_{\rm ZPL} - \hbar \omega_{fn} - \hbar \omega) $

where:
   - $n_r$: index of refraction.
   - $\eta_{\rm sp}$: spin degeneracy factor
        ```
        doublet->(singlet|triplet): 0.5
        singlet->doublet: 1.0
        ```

The capture coefficient is defined as

$C_n = Vr$

where $r$ (the luminescence rate) is the capture rate of one $e^-$ by one defect in a volume V.

The total luminescence rate $r$ can be obtained by integrating over all energies ($\hbar \omega$):

$ r = \int d(\hbar\omega) \, I(\hbar \omega) = \frac{e^2 n_r \eta_{\rm sp} \omega}{3 m^2 \epsilon_0 \pi c^3 \hbar} \sum_n \left| \left< \chi_{i0} | \chi_{fn}  \right> \right|^2 \delta(E_{\rm ZPL} - \hbar \omega_{fn} )$ 



## Renormalizing for simulation cell

The volume defined by $1/N_{A}^0$ is going to be much larger than the size of the simulation cell $\tilde V$.

$C_{n} = \eta_{\rm sp}$

The prefactor should be

$\frac{2}{3}\frac{e^2}{m \hbar ^ 2 c^3 \epsilon_0}$

In [ ]:
from scipy.constants import hbar, c, epsilon_0, e, pi, electron_mass, eV
2/3 *e**2/(hbar**2 * c**3 * epsilon_0 * electron_mass) 

7.080719200902919e+45

Bohr radius in meters: 5.29177 x 10 -11

$\frac{4 \pi \epsilon_0 \hbar^2}{m_e e^2}$